In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
#!pip install --upgrade keras

In [3]:
import pandas as pd
import numpy as np
import nltk

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization



In [4]:
reviews = pd.read_csv('https://raw.githubusercontent.com/nursnaaz/GoogletoChatgpt/main/05.%20Building%20Text%20Classification/Sentiment%20Analysis/movie_reviews.csv')

In [5]:
reviews.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [6]:
reviews.sample(3000).sentiment.value_counts()

negative    1530
positive    1470
Name: sentiment, dtype: int64

In [7]:
reviews_sample = reviews.sample(3000)

In [8]:
reviews_sample.sentiment.value_counts()

positive    1523
negative    1477
Name: sentiment, dtype: int64

In [9]:
reviews_sample = reviews_sample.reset_index().drop(columns = 'index')

In [10]:
reviews_sample

,review,sentiment
0,I'm a Black man living in a predominantly Blac...,negative
1,"I saw this a good while ago, but i just cant g...",positive
2,This is one of the silliest movies I have ever...,negative
3,In the unlikely case that some aspiring direct...,negative
4,I gave this two stars for the awesome DV shot ...,negative
...,...,...
2995,I have wanted to see this for the longest time...,positive
2996,This movie down-shifts from 4th into 1st witho...,negative
2997,This is a Japanese film but there is quite a b...,negative
2998,This is a very good movie. Do you want to know...,positive


In [11]:
import nltk
import re
from bs4 import BeautifulSoup

stop_words = nltk.corpus.stopwords.words('english')


def strip_html(doc):
    soup = BeautifulSoup(doc,"html.parser")
    text = soup.get_text()
    return text


def normalize_document(doc):
    doc = strip_html(doc)
    # lower case and remove special characters\whitespaces
    #doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    #filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    #doc = ' '.join(filtered_tokens)
    doc = ' '.join(tokens)
    return doc

In [12]:
reviews_sample['review_sample'] = reviews_sample['review'].apply(normalize_document)

In [13]:
reviews_sample

,review,sentiment,review_sample
0,I'm a Black man living in a predominantly Blac...,negative,i 'm a black man living in a predominantly bla...
1,"I saw this a good while ago, but i just cant g...",positive,"i saw this a good while ago , but i just cant ..."
2,This is one of the silliest movies I have ever...,negative,this is one of the silliest movies i have ever...
3,In the unlikely case that some aspiring direct...,negative,in the unlikely case that some aspiring direct...
4,I gave this two stars for the awesome DV shot ...,negative,i gave this two stars for the awesome dv shot ...
...,...,...,...
2995,I have wanted to see this for the longest time...,positive,i have wanted to see this for the longest time...
2996,This movie down-shifts from 4th into 1st witho...,negative,this movie down-shifts from 4th into 1st witho...
2997,This is a Japanese film but there is quite a b...,negative,this is a japanese film but there is quite a b...
2998,This is a very good movie. Do you want to know...,positive,this is a very good movie . do you want to kno...


In [14]:
X = reviews_sample['review_sample']
y = reviews_sample['sentiment']

In [15]:
max_features = 2000
Encoder = TextVectorization( max_tokens = max_features)
Encoder.adapt(X.values)

vocab = np.array(Encoder.get_vocabulary())
print(vocab[:20])

example ="This is an example to test the encoder that we just created!"
print(Encoder(example).numpy())
print(" ".join(vocab[Encoder(example).numpy()]))

['' '[UNK]' 'the' 'and' 'a' 'of' 'to' 'is' 'it' 'in' 'i' 'this' 'that' 's'
 'was' 'as' 'with' 'for' 'movie' 'but']
[  11    7   35  446    6    1    2    1   12   73   42 1088]
this is an example to [UNK] the [UNK] that we just created


In [16]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, )
tokenizer.fit_on_texts(X.values)
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X, padding = 'post' ,maxlen=300)
Y = pd.get_dummies(y).values

vocab_size = len(tokenizer.word_index)+1


In [17]:
train_X, test_X, train_y, test_y = train_test_split(X,Y, test_size=0.33, random_state=23)

In [18]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((2010, 300), (990, 300), (2010, 2), (990, 2))

In [27]:
train_X

array([[ 517,  613,  225, ...,    0,    0,    0],
       [   6,    1,   69, ...,    0,    0,    0],
       [1192,   47,   98, ...,    0,    0,    0],
       ...,
       [   1,  762,   17, ...,    6,    3,  162],
       [  59,   52, 1180, ...,    1,  195,  245],
       [   9,  981,   17, ...,    0,    0,    0]], dtype=int32)

In [35]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Embedding
from keras import optimizers

In [36]:
def deep_lstm():
    embid_dim = 300
    lstm_out = 128
    model = Sequential()
    model.add(Embedding(max_features, embid_dim, input_length = X.shape[1]))
    model.add(LSTM(20, dropout=0.2, return_sequences = True))
    model.add(LSTM(20, dropout=0.2, return_sequences = True))
    model.add(LSTM(20, dropout=0.2, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    adam = optimizers.Adam()
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [37]:
deep_lstm().summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 300)          600000    
                                                                 
 lstm_13 (LSTM)              (None, 300, 20)           25680     
                                                                 
 lstm_14 (LSTM)              (None, 300, 20)           3280      
                                                                 
 lstm_15 (LSTM)              (None, 300, 20)           3280      
                                                                 
 lstm_16 (LSTM)              (None, 20)                3280      
                                                                 
 dense_4 (Dense)             (None, 2)                 42        
                                                                 
 activation_4 (Activation)   (None, 2)                

In [38]:
model = deep_lstm()
model.fit(train_X, train_y, epochs = 10, batch_size = 100, verbose = 1)

Epoch 1/10
21/21 [==============================] - 30s 1s/step - loss: 0.6936 - accuracy: 0.5045
Epoch 2/10
21/21 [==============================] - 20s 933ms/step - loss: 0.6930 - accuracy: 0.5060
Epoch 3/10
21/21 [==============================] - 21s 990ms/step - loss: 0.6925 - accuracy: 0.5189
Epoch 4/10
21/21 [==============================] - 21s 995ms/step - loss: 0.6776 - accuracy: 0.5413
Epoch 5/10
21/21 [==============================] - 20s 940ms/step - loss: 0.6536 - accuracy: 0.5562
Epoch 6/10
21/21 [==============================] - 21s 996ms/step - loss: 0.6205 - accuracy: 0.5905
Epoch 7/10
21/21 [==============================] - 20s 936ms/step - loss: 0.5876 - accuracy: 0.5876
Epoch 8/10
21/21 [==============================] - 21s 993ms/step - loss: 0.5737 - accuracy: 0.5881
Epoch 9/10
21/21 [==============================] - 21s 1s/step - loss: 0.5689 - accuracy: 0.6010
Epoch 10/10
21/21 [==============================] - 23s 1s/step - loss: 0.5682 - accuracy: 0.590

In [41]:
y_pred = model.predict(test_X)
y_test_ = np.argmax(y_pred, axis = 1)

31/31 [==============================] - 3s 105ms/step


In [42]:

print(accuracy_score(np.argmax(test_y, axis = 1), y_test_))

0.4909090909090909


In [43]:
from keras.layers import Bidirectional

In [44]:
def bidirectional_lstm():
    embid_dim = 300
    lstm_out = 128
    model = Sequential()
    model.add(Embedding(max_features, embid_dim, input_length = X.shape[1]))
    model.add(Bidirectional(LSTM(20, return_sequences = False)))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    adam = optimizers.Adam()
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [46]:
model = bidirectional_lstm()
model.fit(train_X, train_y, epochs = 10, batch_size = 1, verbose = 1)

Epoch 1/10
2010/2010 [==============================] - 462s 226ms/step - loss: 0.6665 - accuracy: 0.6035
Epoch 2/10
2010/2010 [==============================] - 423s 210ms/step - loss: 0.4781 - accuracy: 0.7831
Epoch 3/10
2010/2010 [==============================] - 420s 209ms/step - loss: 0.2907 - accuracy: 0.8841
Epoch 4/10
2010/2010 [==============================] - 418s 208ms/step - loss: 0.1667 - accuracy: 0.9343
Epoch 5/10
2010/2010 [==============================] - 421s 209ms/step - loss: 0.0911 - accuracy: 0.9672
Epoch 6/10
2010/2010 [==============================] - 415s 206ms/step - loss: 0.0498 - accuracy: 0.9856
Epoch 7/10
2010/2010 [==============================] - 415s 207ms/step - loss: 0.0356 - accuracy: 0.9876
Epoch 8/10
2010/2010 [==============================] - 417s 207ms/step - loss: 0.0206 - accuracy: 0.9935
Epoch 9/10
2010/2010 [==============================] - 417s 208ms/step - loss: 0.0251 - accuracy: 0.9910
Epoch 10/10
2010/2010 [=======================

In [47]:
y_pred = model.predict(test_X)
y_test_ = np.argmax(y_pred, axis = 1)

31/31 [==============================] - 3s 77ms/step


In [48]:
print(accuracy_score(np.argmax(test_y, axis = 1), y_test_))

0.6777777777777778


In [ ]:
def deep_bidirectional_lstm():
    embid_dim = 300
    lstm_out = 128
    model = Sequential()
    model.add(Embedding(max_features, embid_dim, input_length = X.shape[1]))
    model.add(Bidirectional(LSTM(10, dropout=0.2,return_sequences = True)))
    model.add(Bidirectional(LSTM(10, dropout=0.2,return_sequences = True)))
    model.add(Bidirectional(LSTM(10, dropout=0.2,return_sequences = True)))
    model.add(Bidirectional(LSTM(10, dropout=0.2,return_sequences = False)))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    adam = optimizers.Adam()
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [ ]:
model = deep_bidirectional_lstm()
model.fit(train_X, train_y, epochs = 10, batch_size = 100, verbose = 1)

Epoch 1/10
21/21 [==============================] - 33s 777ms/step - loss: 0.6938 - accuracy: 0.4975
Epoch 2/10
21/21 [==============================] - 16s 752ms/step - loss: 0.6937 - accuracy: 0.4990
Epoch 3/10
21/21 [==============================] - 16s 776ms/step - loss: 0.6934 - accuracy: 0.5045
Epoch 4/10
21/21 [==============================] - 17s 775ms/step - loss: 0.6931 - accuracy: 0.5080
Epoch 5/10
21/21 [==============================] - 16s 773ms/step - loss: 0.6930 - accuracy: 0.4940
Epoch 6/10
10/21 [=============>................] - ETA: 10s - loss: 0.6914 - accuracy: 0.5020

In [ ]:
y_pred = model.predict(test_X)
y_test_ = np.argmax(y_pred, axis = 1)

In [ ]:
print(accuracy_score(np.argmax(test_y, axis = 1), y_test_))